In [6]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function, division
%matplotlib inline

# Demo Illustris Galaxy Shapes

In this notebook, we load the stellar particles from a few galaxies in an Illustris simulation.  Then, using some pre-computed shape measurements, I rotate each galaxy so that it is axis-aligned, i.e. the major, intermediate, and minor axes are aligned with the x,y, and z cartesian axes.  A plot of this image is used in the README for this project. 

## Simulation Data

Here I specify which simulation to examine.

In [7]:
# specify simulation
simname = 'Illustris-1'
snapNum = 99
shape_type = 'iterative'

In [8]:
# load simulation information
from Illustris_Shapes.simulation_props import sim_prop_dict
d = sim_prop_dict[simname]
basePath = d['basePath']
Lbox = d['Lbox']

# load pre-computed shape catalog
from astropy.table import Table
fname = simname + '_' + str(snapNum) + '_' + shape_type + '_' +'galaxy_shapes'+ '.dat' 
t_1 = Table.read('../data/shape_catalogs/'+fname, format='ascii')

## Utility Functions

In this section, I define a function to apply a rotation matrix to a set of particle coordinates given the eigenvectors that define the orientation of a galaxy.  You will need the `illustris_python` and `rotations` packages. 

I also define a function to make a density-weighted porojected image of a galaxy.

In [9]:
from Illustris_Shapes.calculate_galaxy_shapes import format_particles, particle_selection, galaxy_center
from illustris_python.snapshot import loadHalo, snapPath, loadSubhalo
from illustris_python.groupcat import gcPath, loadHalos, loadSubhalos
from rotations.rotations3d import rotation_matrices_from_basis
from rotations import rotate_vector_collection

In [10]:
def axis_aligned_ptcl_dist(gal_id, Av, Bv, Cv):
    """
    return the axis aligned stellar partiucle distribution of a galaxy
    """
    fields = ['SubhaloGrNr', 'SubhaloMassInRadType', 'SubhaloPos', 'SubhaloHalfmassRadType']
    galaxy_table = loadSubhalos(basePath, snapNum, fields=fields)
    gal_position = galaxy_center(gal_id, galaxy_table)
    
    
    # load stellar particle positions and masses
    ptcl_coords = loadSubhalo(basePath, snapNum, gal_id, 4, fields=['Coordinates'])/1000.0
    ptcl_masses = loadSubhalo(basePath, snapNum, gal_id, 4, fields=['Masses'])*10.0**10

    # center and account for PBCs
    ptcl_coords = format_particles(gal_position, ptcl_coords, Lbox)
    
    # make selection
    ptcl_mask = particle_selection(gal_id, ptcl_coords, galaxy_table,
                                   basePath, snapNum, radial_mask=False)
    
    # exrtract particles for the galaxy
    ptcl_coords = ptcl_coords[ptcl_mask]
    ptcl_masses = ptcl_masses[ptcl_mask]
    
    # build a rotation matrix
    rot = rotation_matrices_from_basis([Av], [Bv], [Cv])
    rot = rot[0].T
    
    # rotate the particles to be axis-aligned
    ptcl_coords = rotate_vector_collection(rot, ptcl_coords)
    
    # get the half mass radius
    gal_rhalfs = loadSubhalos(basePath, snapNum, fields=['SubhaloHalfmassRadType'])[:,4]/1000.00
    gal_rhalf = gal_rhalfs[gal_id]
    
    # scale coordinates by the half-mass radius
    return ptcl_coords/gal_rhalf, ptcl_masses


def major_minor_image(ptcl_coords, ptcl_masses, log_density=True):
    """
    Return a surface density weighted image of a galaxy.  
    The galaxy is projected along the y-axis.   
    """
    bins = np.linspace(-2,2,50)
    with np.errstate(divide='ignore'):
        if log_density:
            counts_xz = np.log10(np.histogram2d(ptcl_coords[:,0], ptcl_coords[:,2],
                                                bins=bins, weights=ptcl_masses)[0])
            # mask out 
            mask = (im==-1.0*np.inf)
            im[mask] = np.min(im)
        else:
            counts_xz = np.histogram2d(ptcl_coords[:,0], ptcl_coords[:,2],
                                       bins=bins, weights=ptcl_masses)[0]
    
    # rotate image orientation
    im = counts_xz.T
    
    return im

## Select Galaxies

Define the eigenvectors for each galaxy.

In [11]:
Av = np.vstack((t_1['av_x'], t_1['av_y'], t_1['av_z'])).T
Bv = np.vstack((t_1['bv_x'], t_1['bv_y'], t_1['bv_z'])).T
Cv = np.vstack((t_1['cv_x'], t_1['cv_y'], t_1['cv_z'])).T
ids = t_1['gal_id']

Select galaxies with a stellar mass greater than $10^{10.5} M_{\odot}$.

In [ ]:
# make selection
galaxy_table = loadSubhalos(basePath, snapNum, fields=['SubhaloGrNr', 'SubhaloMassInRadType'])
gal_ids = np.arange(0,len(galaxy_table['SubhaloGrNr']))

# mass of stellar particles within 2*R_half
mstar = galaxy_table['SubhaloMassInRadType'][:,4]
mstar = mstar*10**10

mask = (mstar > 10**10.5)

from halotools.utils import crossmatch
idx, idy = crossmatch(ids,gal_ids[mask])

## Plot Galaxy Images

Here we randomly select five galaxies from the selection made in the previous block.  Then for each galaxy, we axis-align the particles and create an image of the galaxy.  Over-plotted on each image is the projected shape ellipsoid and the half-mass circular radius.   

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Ellipse

inds = np.random.random_integers(0, high=len(idx), size=5)
inds = idx[inds]

fig, axes = plt.subplots(1,5, figsize=(3.3*5,3.3))

# panel 1
i = inds[0]
ptcl_coords, ptcl_masses = axis_aligned_ptcl_dist(ids[i], Av[i], Bv[i], Cv[i])
im = major_minor_image(ptcl_coords, ptcl_masses)
a, c = t_1['a'][i], t_1['c'][i]

ax=axes[0]
ax.set_title('galaxy ID = ' + str(ids[i]))
ax.imshow(im, extent=[-2.5,2.5,-2.5,2.5], origin='lower', vmin=5)
ellipse_0 = Ellipse([0,0], 2*a, 2*c, angle=0, facecolor='none', edgecolor='red')
ax.add_artist(ellipse_0)
ellipse_0 = Ellipse([0,0], 2*2, 2*2, angle=0, facecolor='none', edgecolor='white', linestyle='--')
ax.add_artist(ellipse_0)
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])
ax.set_xlabel(r'$x/r_{1/2}$')
ax.set_ylabel(r'$z/r_{1/2}$')

# panel 2
i = inds[1]
ptcl_coords, ptcl_masses = axis_aligned_ptcl_dist(ids[i], Av[i], Bv[i], Cv[i])
im = major_minor_image(ptcl_coords, ptcl_masses)
a, c = t_1['a'][i], t_1['c'][i]

ax=axes[1]
ax.set_title('galaxy ID = ' + str(ids[i]))
ax.imshow(im, extent=[-2.5,2.5,-2.5,2.5], origin='lower', vmin=5)
ellipse_0 = Ellipse([0,0], 2*a, 2*c, angle=0, facecolor='none', edgecolor='red')
ax.add_artist(ellipse_0)
ellipse_0 = Ellipse([0,0], 2*2, 2*2, angle=0, facecolor='none', edgecolor='white', linestyle='--')
ax.add_artist(ellipse_0)
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])
ax.set_xlabel(r'$x/r_{1/2}$')

# panel 3
i = inds[2]
ptcl_coords, ptcl_masses = axis_aligned_ptcl_dist(ids[i], Av[i], Bv[i], Cv[i])
im = major_minor_image(ptcl_coords, ptcl_masses)
a, c = t_1['a'][i], t_1['c'][i]

ax=axes[2]
ax.set_title('galaxy ID = ' + str(ids[i]))
ax.imshow(im, extent=[-2.5,2.5,-2.5,2.5], origin='lower', vmin=5)
ellipse_0 = Ellipse([0,0], 2*a, 2*c, angle=0, facecolor='none', edgecolor='red')
ax.add_artist(ellipse_0)
ellipse_0 = Ellipse([0,0], 2*2, 2*2, angle=0, facecolor='none', edgecolor='white', linestyle='--')
ax.add_artist(ellipse_0)
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])
ax.set_xlabel(r'$x/r_{1/2}$')

# panel 4
i = inds[3]
ptcl_coords, ptcl_masses = axis_aligned_ptcl_dist(ids[i], Av[i], Bv[i], Cv[i])
im = major_minor_image(ptcl_coords, ptcl_masses)
a, c = t_1['a'][i], t_1['c'][i]

ax=axes[3]
ax.set_title('galaxy ID = ' + str(ids[i]))
ax.imshow(im, extent=[-2.5,2.5,-2.5,2.5], origin='lower', vmin=5)
ellipse_0 = Ellipse([0,0], 2*a, 2*c, angle=0, facecolor='none', edgecolor='red')
ax.add_artist(ellipse_0)
ellipse_0 = Ellipse([0,0], 2*2, 2*2, angle=0, facecolor='none', edgecolor='white', linestyle='--')
ax.add_artist(ellipse_0)
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])
ax.set_xlabel(r'$x/r_{1/2}$')

# panel 5
i = inds[4]
ptcl_coords, ptcl_masses = axis_aligned_ptcl_dist(ids[i], Av[i], Bv[i], Cv[i])
im = major_minor_image(ptcl_coords, ptcl_masses)
a, c = t_1['a'][i], t_1['c'][i]

ax=axes[4]
ax.set_title('galaxy ID = ' + str(ids[i]))
ax.imshow(im, extent=[-2.5,2.5,-2.5,2.5], origin='lower', vmin=5)
ellipse_0 = Ellipse([0,0], 2*a, 2*c, angle=0, facecolor='none', edgecolor='red')
ax.add_artist(ellipse_0)
ellipse_0 = Ellipse([0,0], 2*2, 2*2, angle=0, facecolor='none', edgecolor='white', linestyle='--')
ax.add_artist(ellipse_0)
ax.set_xlim([-2.5, 2.5])
ax.set_ylim([-2.5, 2.5])
ax.set_xlabel(r'$x/r_{1/2}$')

plt.show()

#fig.savefig('./figures/demo_shapes.png')